## import local

if no test dataset:
    train,test    =split
 = Balance(train)

clf = RandomForestClassifier(n_estimators=50, n_jobs=-1)

bagging n boost = True

sklearn.calibration.
clf_cal = CalibratedClassifierCV(cv_clf,cv=prefit, cv = k)   resulting in k**2

2 modes for kaggle
1. train,train-valid, train-test using split-train_test
    ratio = .2 verify model against train dataset
2. train,train-valid, test using split-train_test 
   create model from bigger trainset, using ratio = 0.2.0,1 etce
use clf_cal. predict on test


In [1]:
!conda list

# packages in environment at /Users/brucecottman/anaconda3/envs/paso:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
absl-py                   0.7.0                    pypi_0    pypi
alabaster                 0.7.12                   py37_0  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.6                    py37_0  
anaconda-project          0.8.2                    py37_0  
ansimarkup                1.4.0                    pypi_0    pypi
appnope                   0.1.0                    py37_0  
appscript                 1.0.1            py37h1de35cc_1  
asn1crypto                0.24.0                   py37_0  
astor                     0.7.1                    pypi_0    pypi
astroid                   2.1.0                    py37_0  
astropy                   3.1              py37h1de35cc_0  
atomicwrites              1.2.1                    py37_0  
att

In [2]:
from __future__ import print_function
__file__ = !cd .. ;pwd
__file__ = __file__[0]
__file__

'/Users/brucecottman/Documents/PROJECTS/paso'

In [3]:
import sys
from random import random
import pandas as pd
from tqdm import tqdm

sys.path.append(__file__)
sys.path

['/Users/brucecottman/Documents/PROJECTS/paso/lessons',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python37.zip',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/lib-dynload',
 '',
 '/Users/brucecottman/.local/lib/python3.7/site-packages',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages/aeosa',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages/IPython/extensions',
 '/Users/brucecottman/.ipython',
 '/Users/brucecottman/Documents/PROJECTS/paso']

In [4]:
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary

import warnings
warnings.filterwarnings("ignore")
import multiprocessing

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import seaborn as sns
color = sns.color_palette()
sns.set(rc={'figure.figsize':(6,4)})
from paso.base import Paso,Log,PasoError
from loguru import logger
session = Paso(parameters_filepath='../parameters/lesson.3.yaml').startup()

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely
paso 17.8.2019 17:04:12 INFO Log started
paso 17.8.2019 17:04:12 INFO ========================================
paso 17.8.2019 17:04:12 INFO Read in parameter file: ../parameters/lesson.3.yaml


In [5]:
from paso.base import Paso,Log,PasoError,Param,NameToClass
from loguru import logger
session = Paso(parameters_filepath='../parameters/lesson.3.yaml').startup()

paso 17.8.2019 17:04:12 INFO ========================================
paso 17.8.2019 17:04:12 INFO Read in parameter file: ../parameters/lesson.3.yaml


As we saw in [lesson-1](), we need to startup **paso** services.

In [6]:
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary

import warnings
warnings.filterwarnings("ignore")
import multiprocessing

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import seaborn as sns
color = sns.color_palette()

from paso.base import Paso,Log,PasoError
from loguru import logger
session = Paso(parameters_filepath='../parameters/lesson.3.yaml').startup()

paso 17.8.2019 17:04:12 INFO ========================================
paso 17.8.2019 17:04:12 INFO Read in parameter file: ../parameters/lesson.3.yaml


## Toy Example Dataset

The **paso** class is a general method to read in datasets and return them as dataframes. For now, we can use them. In a future lesson we will cover  ``Inputer`` in  detail.  All available ``Inputer`` methods are shown with:

In [7]:
from paso.pre.inputers import Inputer,Splitter
dataset_name = 'yeast3'
inputer = Inputer(ontological_filepath='../ontologies/pre/inputers/'+dataset_name+'.yaml')
inputer.inputers()

['exec', 'cvs', 'xls', 'xlsm', 'text', 'image2D', 'image3D']

In [8]:
inputer.datasets()

['train', 'valid', 'test', 'sampleSubmission', 'directory_path']

Let us start with a toy dataset to class balance. we load the ``iris``data set, into the ``flower`` dataframe.  The feature ``TypeOf`` will contain the class target values.

In [9]:
yeast3 = inputer.transform()
inputer.target

paso 17.8.2019 17:04:12 INFO Loaded Ontological file:../ontologies/pre/inputers/yeast3.yaml 
paso 17.8.2019 17:04:12 INFO 
 kind-name:cvs
 description:yeast3 dataset
 genus: Inputer
 type: dataset
 kwargs: {'target': 'Class', 'directory_path': '/Users/brucecottman/Documents/PROJECTS/paso/data/', 'train': 'yeast3.csv'}


'Class'

In [10]:
yeast3[inputer.target]= pd.Categorical(yeast3[inputer.target])
yeast3[inputer.target]= yeast3[inputer.target].cat.codes

### A short aside on ontological files.

Instead of using keyword arguments cemented in code, I will use an ontological file. One working definition of ontological (from Wikipedia) is:

    "showing the relations between the concepts and categories in a subject area or domain".

We are using ontological files because:
- the description of an object (dataset, model,etc.) can be changed without changing code
- the parsing performance of an ontological file and python are about the same as both use dynamic typing. This probably would not be case for statically typed language. However, an object's metadata parsing compute is very tiny compared to the action(s) performed on or by the object in all cases encountered so far.  

We will found out if ontological files have a measurable overhead as we port **paso** to other frameworks (such as **RAPIDS**) or statically-typed languages (such as ``swift``). At much of python's workhorse packages are in fast, statically-typed ``C``, so I doubt it.

Ontological files consist of hierarchy of key-value pairs.  In a future lesson we will cover Ontological files in  detail. [Different Ontological files can be viewed here.](https://github.com/bcottman/paso/tree/master/ontologies)


In [11]:
DataFrameSummary(yeast3).summary()

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,Class
count,1484,1484,1484,1484,1484,1484,1484,1484,1484
mean,0.500121,0.499933,0.500034,0.261186,0.504717,0.0075,0.499885,0.276199,0.109838
std,0.137299,0.123924,0.0866702,0.137098,0.048351,0.0756827,0.0577966,0.106491,0.312793
min,0.11,0.13,0.21,0,0.5,0,0,0,0
25%,0.41,0.42,0.46,0.17,0.5,0,0.48,0.22,0
50%,0.49,0.49,0.51,0.22,0.5,0,0.51,0.22,0
75%,0.58,0.57,0.55,0.32,0.5,0,0.53,0.3,0
max,1,1,1,1,1,0.83,0.73,1,1
counts,1484,1484,1484,1484,1484,1484,1484,1484,1484
uniques,81,79,53,78,2,3,48,68,2


## It is important to split first before balancing

Before training the learner it is very important to balance only the dataset we will train on. We want to know the effect of balancing data on the learner, while not corrupting our validation  dataset with augmented  data that balances the classes.

Validation is drawn from the initial train dataset in the hopes it is a sample of the past and future data that the test dataset will be drawn from. The other case where train, valid, test are sampled from the original dataset is covered by splitting the initial dataset to result in train and valid. Then split on train again to get train and test. The order does not matter so long as the second split is also on the train dataset .

In [12]:
splitter = Splitter(ontological_filepath='../ontologies/pre/inputers/test_size_30.yaml')
train,test=splitter.transform(yeast3,target=inputer.target)
train.head()

paso 17.8.2019 17:04:13 INFO Loaded Ontological file:../ontologies/pre/inputers/test_size_30.yaml 
paso 17.8.2019 17:04:13 INFO 
 kind-name:TrTest_split
 description:split into train-test dataset
 genus: Splitter
 type: None
 kwargs: {'test_size': 0.3, 'shuffle': True, 'stratify': True, 'random_state': 66}


,Gvh,Alm,Erl,Mcg,Mit,Nuc,Pox,Vac
1371,0.60,0.65,0.5,0.31,0.18,0.22,0.0,0.46
563,0.41,0.58,0.5,0.45,0.48,0.64,0.0,0.53
1142,0.63,0.45,0.5,0.50,0.13,0.32,0.0,0.45
705,0.41,0.47,0.5,0.24,0.16,0.38,0.0,0.51
1259,0.50,0.48,0.5,0.50,0.17,0.26,0.0,0.47


The argument ratio is the amount of the dataset to assign to the validation (test) dataset. In this case,
``X``and ``y`` datasets into training set(50\%) and test set(50\%). 

Note: A ``valid`` dataset is not made as we are not tuning learner parameters in this lesson. 

## paso Class for Handling Imbalanced Classes

All available class balance strategies are shown with:

In [13]:
from paso.pre.cleaners import Class_Balance
class_balancer = Class_Balance(ontological_filepath='../ontologies/cleaners/SMOTE.yaml')
class_balancer.classBalancers()

['RanZOverSample',
 'SMOTE',
 'ADASYN',
 'BorderLineSMOTE',
 'SVMSMOTE',
 'SMOTENC',
 'RandomUnderSample',
 'ClusterCentroids',
 'NearMiss',
 'EditedNearestNeighbour',
 'RepeatedEditedNearestNeighbours',
 'CondensedNearestNeighbour',
 'OneSidedSelection']

I am going to oversample the minority classes by using ``SMOOT``. I reccomend [Handling imbalanced datasets in machine learning](https://towardsdatascience.com/handling-imbalanced-datasets-in-machine-learning-7a0e84220f28) for more detail on how to balance class imbalanced datasets.

In [14]:
targetFeature = inputer.target
class_balancer = Class_Balance(ontological_filepath='../ontologies/pre/cleaners/SMOTE.yaml')
train = class_balancer.transform(train,target=inputer.target)

NameError: name '_boiler_plate' is not defined

In [ ]:
DataFrameSummary(train).summary()

``Class_Balance`` did not change the ``Flower-train`` dataframe as the ``iris``dataset comes class balanced. 

In [ ]:
train[['Mcg',inputer.target]].groupby([inputer.target]).count()

In [ ]:
test[['Mcg',inputer.target]].groupby([inputer.target]).count()

## A short aside on cross-validation and classification train/predict model

In an future lesson, I will cover in detail how to use **paso**'s cross-validation for learners that can train and then predict. In the meantime, you can find a good [overview of cross-validation](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and a [detailed discussion of cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html#computing-cross-validated-metrics).

In this lesson, we use **repeated random sub-sampling** or ``sklearn.ensemble.BaggingClassifier``. This is hopefully a good choice for the class stratified datasets we will be using.

Similar to **K-Fold cross-validation** , we set a value for ``n_estimators`` which signifies the number of times we will train our learner on a random subset of the train dataset. However, in this case ``n_estimator``  will not represent the number of learners.
Instead, on each training iteration, we randomly select data points for the train folds. The number of data points we select will be a certain percentage we set for the initial trainng set. For example, if we select ``max_samples= 0.90``, then each iteration we will apply the learner to randomly selected 90% of the data points of our original train dataset.  This is similar to ``K-fold`` with ``K=10``, except the samples in folds are chosen at random and it will be more computational expensive if  ``n_estimator >> K``.

 You should note that ``BaggingClassifier`` will need keep ``n_estimators`` dataset+tree around. If you have a large dataset, it maybe required to pare the dataset down by removing the weakest predictive features in the initial cleaning. I should have done this with the **otto-group**, but I did not. 

## Using the RandomForest learner

In [ ]:
from paso.learners.learners import Learner
learner = Learner(ontological_filepath='../ontologies/learners/RandomForestClassification.sm.yaml',target='TypeOf')
learner.train(train,target=inputer.target
              ,checkpoint='diabetesRandomForest1.ckp')

Because the population of each of the Iris classes is about the same, the accuracy and the F1-score are almost equal.

In [ ]:
learner.predict(test,measure=True)
learner.metrics

In [ ]:
learner.wrong_predicted_class,learner.wrong_predicted_class.shape

Overall, very good scores using ``RandomForestClassifier`` on 70% of dataset train and 30% for of dataset for validation.

Interesting there were correct class assignments but logloss got slightly worse when we balanced the classes using ``SMOTE``.

Using splitter, we can shuffle rows around and get different ``piam-diabetes`` train and test datasets.

## Augment the ``yeast3``dataset.

Now we can augment (increase by 10\% the amount data) with  synthetic data generated by a claas imbalence scheme,  ``SMOOT``. Just ``TypeOf=0`` needs 15(10%) rows (3 classes of 5 rows) of synthetic data to balance the classes. The result is similar to image augmention in that we accomplished structured data augmentation by increasing the row count ``Isis``dataset with synthetic data.

In [ ]:
from paso.pre.cleaners import Augment_by_Class
augmenter = Augment_by_Class(ontological_filepath='../ontologies/pre/cleaners/SMOTE.yaml')

augment =augmenter.transform(train,target=inputer.target,ratio=0.5)
DataFrameSummary(augment).summary()

Notice the ``mean``, ``std`` and other statistics have slightly changed. Also, the count has inreased by 10% or augmented a structured dataset by 10%. Why do expect slight and not large change in the dataset statistics?

## Train , Predict Augmented Dataset

In [ ]:
learner.train(augment,target=inputer.target)
learner.predict(test,measure=True)
learner.metrics

Hmmm, seemed to improve scores a little bit. Let us try 100% augmentation or double the number of rows in the  ``Flower`` dataframe.

In [ ]:
learner.wrong_predicted_class.shape

## Augmentation ``pima-diabetes`` train and test datasets

In [ ]:
def grinder(N,M,S,input_file):
    score = np.zeros((S,N,M+2), dtype=np.float32)
    scores = {}           

    for i in tqdm(range(N)):
        inputer = Inputer(ontological_filepath=input_file)
        df = inputer.transform(verbose=False)
        df[inputer.target]= pd.Categorical(df[inputer.target])
        df[inputer.target]= df[inputer.target].cat.codes
        splitter = Splitter(ontological_filepath='../ontologies/pre/inputers/split-stratify-shuffle-30.yaml')
        train,test=splitter.transform(df,verbose=False
                                      ,target=inputer.target
                                      ,random_state=22*(i+1))
        # do not balancee       
        learner = Learner(ontological_filepath='../ontologies/learners/RandomForestClassification.sm.yaml',target='TypeOf')
        learner.train(train,target=inputer.target,verbose=False
                      ,checkpoint='diabetesRandomForest1.ckp')  
        learner.predict(test,measure=True)
        score[0,i,0] = learner.metrics['logloss']
        score[1,i,0] = learner.metrics['AOC']
        #Class_Balance
        class_balancer = Class_Balance(ontological_filepath='../ontologies/pre/cleaners/SMOTE.yaml')
        train = class_balancer.transform(train,target=inputer.target,verbose=False)
        learner = Learner(ontological_filepath='../ontologies/learners/RandomForestClassification.pima.yaml',target='TypeOf')
        learner.train(train,target=inputer.target,verbose=False
                      ,checkpoint='diabetesRandomForest2.ckp')
        learner.predict(test,measure=True)
        score[0,i,1] = learner.metrics['logloss']
        score[1,i,1] = learner.metrics['AOC']
        #augment
        augment = train
        for j in range(M):
        # 2*1 augmentation
            augmenter = Augment_by_Class(ontological_filepath='../ontologies/pre/cleaners/SMOTE.yaml')
            ratio = 1.0
            targetFeature =  inputer.target
            augment =augmenter.transform(augment,target=inputer.target,ratio=1.0,verbose=False)
            learner = Learner(ontological_filepath='../ontologies/learners/RandomForestClassification.sm.yaml',target='TypeOf')
            learner.train(augment,target=inputer.target,verbose=False
                          ,checkpoint='diabetesRandomForest'+str(i+2)+'.ckp')
            learner.predict(test,measure=True)
            score[0,i,j+2] = learner.metrics['logloss']
            score[1,i,j+2] = learner.metrics['AOC']
        learner.wrong_predicted_class
    scores['logloss'] = score[0,:,:]
    scores['AOC'] = score[1,:,:]
    return scores

In [ ]:
N = 10  #trial
M = 4 # augmentation 2**M
S= 2 #scores to keep
scores = grinder(N,M,S,'../ontologies/pre/inputers/yeast3.yaml')

In [ ]:
baseline = scores['logloss'][:,0]
aug_balance = np.mean(scores['logloss'][:,1:2],axis=1)
baseline,aug_balance


In [ ]:
np.mean(baseline/aug_balance)

In [ ]:
sns.scatterplot(x=baseline ,y=baseline/aug_balance)

In [ ]:
import seaborn
for i in range(N):
    grid = seaborn.lineplot(x=range(M+2) ,y=scores['logloss'][i,:])
    grid.set(xscale="linear", yscale="log")
plt.show()
for i in range(N):
    grid = seaborn.lineplot(x=range(M+2) ,y=scores['AOC'][i,:])
    grid.set(xscale="linear", yscale="log")
plt.show()

It seems to be more worse on logloss but same on other metrics relating to class assignment.

Worse on logloss and the same on accuracy

For the third time in a row always the same 2 datapoints. It seems no matter how much we augment these two can not predicted in the right class.  I wondr if they are mis-labeled.  If we take them out, then train will predict test 100% accuracy. Hmm!, I wonder what logloss score would be?

Seemed to lower the logloss more which means the learner is better at assigning correct class probabilities. Note,isotonic in ``CalibratedClassifierCV`` is similar to clipping both ends of predicted class probability distributions. Let us augment the train dataset again.

Still causing logloss to go down. Let us stop there on this toy dataset and see how we do on a larger and noisier dataset. 

Now we cleanup all the object instances we made and no longer need:

In [ ]:
del inputer
del splitter
del class_balancer
del augmenter
del learner
del yeast3
del train
del test
del augment